In [1]:
!pip install ultralytics --quiet --no-deps

import gc
import yaml
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path

gc.collect()
torch.cuda.empty_cache()

data_config = {
    "path": "/kaggle/input/spotify-augmented/spotify-augmented-v3",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": 1,
    "names": ["spotify_logo"],
}

output_yaml = "/kaggle/working/data_fixed.yaml"
with open(output_yaml, "w") as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✓ Створено новий data.yaml з правильними шляхами")

base_path = Path("/kaggle/input/spotify-augmented/spotify-augmented/spotify-augmented")
print(f"\n✓ Перевірка структури датасету:")
print(f"  train/images: {(base_path / 'train' / 'images').exists()}")
print(f"  valid/images: {(base_path / 'valid' / 'images').exists()}")
print(f"  test/images: {(base_path / 'test' / 'images').exists()}")

yolo_model = YOLO("yolo11s-seg.pt")

print("\n=== Початок навчання (SEG) ===")
results = yolo_model.train(
    data=output_yaml,
    epochs=50,
    imgsz=640,
    batch=16,
    workers=4,
    patience=15,
    rect=False,

    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,

    save=True,
    save_period=10,
    plots=True,
    device=0,
    pretrained=True,
    verbose=True,
    name="spotify_segmentation"
)

print(f"\n✓ Навчання завершено!")
print(f"✓ Результати: /kaggle/working/runs/segment/spotify_segmentation")

print("\n=== Валідація SEG-моделі ===")
val_results = yolo_model.val(data=output_yaml)

print(f"\n[BOX] mAP50:     {val_results.box.map50:.4f}")
print(f"[BOX] mAP50-95:  {val_results.box.map:.4f}")

if hasattr(val_results, "seg"):
    print(f"\n[MASK] mAP50:    {val_results.seg.map50:.4f}")
    print(f"[MASK] mAP50-95: {val_results.seg.map:.4f}")

print(f"\n✓ Найкраща модель: runs/segment/spotify_segmentation/weights/best.pt")
print(f"✓ Всі графіки та логи: runs/segment/spotify_segmentation/")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.6 MB/s eta 0:00:00
,Creating new Ultralytics Settings v0.0.6 file ✅ 
,View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
,Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
,✓ Створено новий data.yaml з правильними шляхами
,
,✓ Перевірка структури датасету:
,  train/images: False
,  valid/images: False
,  test/images: False
,Downloading https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s-seg.pt to 'yolo11s-seg.pt': 100% ━━━━━━━━━━━━ 19.7MB 90.2MB/s 0.2s
,
,=== Початок навчання (SEG) ===
,Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
,engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
,  xa[xa < 0] = -1
,/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
,  xa[xa < 0] = -1


                   all         30         59      0.934      0.983      0.991      0.981      0.934      0.983      0.991       0.98
,Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 1.3ms postprocess per image
,Results saved to /kaggle/working/runs/segment/spotify_segmentation
,
,✓ Навчання завершено!
,✓ Результати: /kaggle/working/runs/segment/spotify_segmentation
,
,=== Валідація SEG-моделі ===
,Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
,YOLO11s-seg summary (fused): 113 layers, 10,067,203 parameters, 0 gradients
,val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 39.3±13.9 MB/s, size: 33.0 KB)
,val: Scanning /kaggle/input/spotify-augmented/spotify-augmented-v3/valid/labels... 30 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 30/30 569.3it/s 0.1s
,WARNING ⚠️ val: Cache directory /kaggle/input/spotify-augmented/spotify-augmented-v3/valid is not writable, cache not saved.
,                 Class     Images  Instances      Bo

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
,  xa[xa < 0] = -1
,/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
,  xa[xa < 0] = -1


                   all         30         59      0.929      0.983      0.989       0.98      0.929      0.983      0.989      0.974
,Speed: 3.3ms preprocess, 16.1ms inference, 0.0ms loss, 4.1ms postprocess per image
,Results saved to /kaggle/working/runs/segment/val
,
,[BOX] mAP50:     0.9891
,[BOX] mAP50-95:  0.9802
,
,[MASK] mAP50:    0.9891
,[MASK] mAP50-95: 0.9742
,
,✓ Найкраща модель: runs/segment/spotify_segmentation/weights/best.pt
,✓ Всі графіки та логи: runs/segment/spotify_segmentation/
